#  Data Engineering: COVID-19 USA Testing

The purpose of this notebook is to supplement [COVID-19 case count data](https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6) with COVID-19 testing data from [The COVID Tracking Project](https://covidtracking.com/) for test counts at the US state level and produce a GIS dataset that can be used in applications and downstream analysis. 

### General Process:

1. Ingest Data

2. Prepare Data (Geoenable and Calculate Testing Rates)

3. Export to local feature class

### Setup

In [1]:
import pandas as pd
import os
import requests
from datetime import date
import arcgis
import arcpy

In [2]:
pro_project = arcpy.mp.ArcGISProject("CURRENT")
pro_map = pro_project.listMaps(pro_project.activeMap.name)[0]
default_fgdb = pro_project.defaultGeodatabase

## 1. Ingest Data

In [4]:
api_url = r"https://covidtracking.com/api/states"
r = requests.get(url = api_url)
data = r.json()
tests_df = pd.DataFrame(data)

In [5]:
tests_df.head()

,state,positive,positiveScore,negativeScore,negativeRegularScore,commercialScore,grade,score,negative,pending,hospitalized,death,total,lastUpdateEt,checkTimeEt,dateModified,dateChecked
0,AK,42,1.0,1.0,1.0,1.0,A,4.0,1691.0,NaN,1.0,1.0,1733,3/24 21:00,3/24 22:29,2020-03-25T01:00:00Z,2020-03-25T02:29:00Z
1,AL,242,1.0,1.0,0.0,0.0,C,2.0,2106.0,NaN,NaN,0.0,2348,3/24 17:00,3/24 22:10,2020-03-24T21:00:00Z,2020-03-25T02:10:00Z
2,AR,232,1.0,1.0,1.0,1.0,A,4.0,998.0,0.0,22.0,2.0,1230,3/24 00:00,3/24 21:11,2020-03-24T04:00:00Z,2020-03-25T01:11:00Z
3,AZ,367,1.0,1.0,1.0,0.0,B,3.0,313.0,22.0,8.0,5.0,702,3/24 00:00,3/24 22:13,2020-03-24T04:00:00Z,2020-03-25T02:13:00Z
4,CA,2102,1.0,1.0,1.0,0.0,B,3.0,13452.0,12100.0,NaN,40.0,27654,3/24 00:00,3/24 21:24,2020-03-24T04:00:00Z,2020-03-25T01:24:00Z


## 2. Prepare Data (Geoenable and Calculate Testing Rates)

In [6]:
# Authenticate with a GIS using the ArcGIS API for Python
gis = arcgis.gis.GIS()

In [11]:
# Retrieve USA States geometry data using the Living Atlas layer
states_item = gis.content.get("99fd67933e754a1181cc755146be21ca")
states_item

<Item title:"USA States (Generalized)" type:Feature Layer Collection owner:esri_dm>

In [8]:
# Read the layer into a dataframe
states_df = states_item.layers[0].query().sdf
states_df = states_df[['FID', 
                       'STATE_NAME', 
                       'STATE_FIPS', 
                       'SUB_REGION', 
                       'STATE_ABBR', 
                       'POPULATION', 
                       'SHAPE']]
states_df.head()

,FID,STATE_NAME,STATE_FIPS,SUB_REGION,STATE_ABBR,POPULATION,SHAPE
0,1,Alaska,02,Pacific,AK,744733,"{'rings': [[[-17959594.8053098, 8122953.575198..."
1,2,California,06,Pacific,CA,39611295,"{'rings': [[[-13543710.3257494, 4603367.827345..."
2,3,Hawaii,15,Pacific,HI,1461211,"{'rings': [[[-17819334.303422, 2512026.7784964..."
3,4,Idaho,16,Mountain,ID,1714694,"{'rings': [[[-13027307.5891034, 5415905.134774..."
4,5,Nevada,32,Mountain,NV,2994047,"{'rings': [[[-13263990.1054907, 4637763.931898..."


In [9]:
# Join the data to our testing data table
geo_df = pd.merge(states_df, tests_df, left_on='STATE_ABBR', right_on='state', how='left')
# Visualize the data at this stage
geo_df.head()

,FID,STATE_NAME,STATE_FIPS,SUB_REGION,STATE_ABBR,POPULATION,SHAPE,state,positive,positiveScore,negativeScore,negativeRegularScore,commercialScore,grade,score,negative,pending,hospitalized,death,total,lastUpdateEt,checkTimeEt,dateModified,dateChecked
0,1,Alaska,02,Pacific,AK,744733,"{'rings': [[[-17959594.8053098, 8122953.575198...",AK,42,1.0,1.0,1.0,1.0,A,4.0,1691.0,NaN,1.0,1.0,1733,3/24 21:00,3/24 22:29,2020-03-25T01:00:00Z,2020-03-25T02:29:00Z
1,2,California,06,Pacific,CA,39611295,"{'rings': [[[-13543710.3257494, 4603367.827345...",CA,2102,1.0,1.0,1.0,0.0,B,3.0,13452.0,12100.0,NaN,40.0,27654,3/24 00:00,3/24 21:24,2020-03-24T04:00:00Z,2020-03-25T01:24:00Z
2,3,Hawaii,15,Pacific,HI,1461211,"{'rings': [[[-17819334.303422, 2512026.7784964...",HI,77,1.0,1.0,1.0,1.0,A,4.0,3589.0,NaN,4.0,1.0,3666,3/23 18:00,3/24 21:33,2020-03-23T22:00:00Z,2020-03-25T01:33:00Z
3,4,Idaho,16,Mountain,ID,1714694,"{'rings': [[[-13027307.5891034, 5415905.134774...",ID,73,1.0,1.0,1.0,1.0,A,4.0,1887.0,NaN,NaN,0.0,1960,3/24 19:00,3/24 22:31,2020-03-24T23:00:00Z,2020-03-25T02:31:00Z
4,5,Nevada,32,Mountain,NV,2994047,"{'rings': [[[-13263990.1054907, 4637763.931898...",NV,278,1.0,1.0,1.0,1.0,A,4.0,3954.0,0.0,NaN,4.0,4232,3/23 14:59,3/24 22:45,2020-03-23T18:59:00Z,2020-03-25T02:45:00Z


In [10]:
geo_df['tests_per_1M_residents'] = geo_df['total'] / (geo_df['POPULATION'] / 1000000)

## 3. Export to local feature class

In [26]:
date.today().strftime("%Y%m%d")

# Use API to_featureclass method since we have geometry
fgdb = arcpy.mp.ArcGISProject("CURRENT").defaultGeodatabase
out_fc_name = "covid19_testing_"+date.today().strftime("%Y%m%d")
out_fc = geo_df.spatial.to_featureclass(os.path.join(fgdb, out_fc_name))

out_archive_fc_name = "covid19_archived_testing_"+date.today().strftime("%Y%m%d"+"_temp")
out_archive_fc = geo_df.spatial.to_featureclass(os.path.join(fgdb, out_archive_fc_name))